## Query OpenAlex for works authored by a person
This notebook queries the [OpenAlex API](https://docs.openalex.org/api) via its `/works` endpoint for works authored by a person. It takes an ORCID URL from a list of ORCID IDs as input which is used to filter for works where '`authorships.author.orcid`' matches the given ORCID URL.
The notebook iterates through the given list and displys all DOIs.

## Concept ID to ORCID

Usecase: Expertensuche und Trackrecord

Anmerkungen: Doppelsuche nötig um spezifische Uni zu suchen (ORCID und ROR)
Muss Endpoint geändert werden?

In [1]:
# Prerequisites:
import requests         # dependency to make HTTP calls

In [2]:
# List of all 86 ORCID IDs of members of the University of Osnabrück
list_of_ids=["0000-0001-5380-4449",       
"0000-0001-5406-9458",
"0000-0001-5449-4593",
"0000-0001-5913-890X",
"0000-0001-6604-6253",
"0000-0001-7263-2670",
"0000-0001-7364-4315",
"0000-0001-7389-8024",
"0000-0001-7973-3140",
"0000-0001-8234-9166",
"0000-0001-8307-2189",
"0000-0001-8343-8654",
"0000-0001-8481-6047",
"0000-0001-8498-9466",
"0000-0001-8585-781X",
"0000-0001-9469-2367",
"0000-0002-0256-0680",
"0000-0002-0684-6707",
"0000-0002-0735-5088",
"0000-0002-1187-5166",
"0000-0002-1273-5819",
"0000-0002-1417-2722",
"0000-0002-1424-6314",
"0000-0002-1846-647X",
"0000-0002-2050-9221",
"0000-0002-2143-2270",
"0000-0002-2194-8293",
"0000-0002-2224-4503",
"0000-0002-2456-1174",
"0000-0002-2572-3390",
"0000-0002-2586-3748",
"0000-0002-2747-0913",
"0000-0002-2768-8381",
"0000-0002-2769-0692",
"0000-0002-2845-6945",
"0000-0002-2950-534X",
"0000-0002-3043-3718",
"0000-0002-3108-5217",
"0000-0002-3416-2652",
"0000-0002-3650-1056",
"0000-0002-3796-3500",
"0000-0002-3912-9093",
"0000-0002-4156-3761",
"0000-0002-4398-2337",
"0000-0002-4467-1864",
"0000-0002-4681-5550",
"0000-0002-4789-7084",
"0000-0002-5039-6950",
"0000-0002-5229-0500",
"0000-0002-5241-8498",
"0000-0002-5535-8179",
"0000-0002-5581-7371",
"0000-0002-5861-8896",
"0000-0002-5868-755X",
"0000-0002-6328-7745",
"0000-0002-6371-9624",
"0000-0002-6649-5064",
"0000-0002-6666-1499",
"0000-0002-7366-679X",
"0000-0002-7541-4369",
"0000-0002-7839-6397",
"0000-0002-7870-7343",
"0000-0002-7972-6925",
"0000-0002-8449-1593",
"0000-0002-8722-3332",
"0000-0002-8845-6859",
"0000-0002-9686-8810",
"0000-0003-0608-0884",
"0000-0003-0830-9603",
"0000-0003-0851-2767",
"0000-0003-0858-4760",
"0000-0003-1005-5753",
"0000-0003-1626-0598",
"0000-0003-1813-718X",
"0000-0003-1976-8186",
"0000-0003-2001-6440",
"0000-0003-2162-1968",
"0000-0003-2340-3462",
"0000-0003-2967-2858",
"0000-0003-3186-9000",
"0000-0003-3459-5148",
"0000-0003-3547-3257",
"0000-0003-3654-5267",
"0000-0003-4331-8695",
"0000-0003-4939-1666",
"0000-0003-4971-9991",]

In [3]:
Fachsuche=['Gender studies',
'Gender equality',
'Gender bias',
'Gender history',
'Gender equity',
'Gender schema theory',
'Gender role',
'Gender gap',
'Language and gender',
'Doing gender',
'Gender violence',
'Gender justice',
'Transgender',
'Gender relations',
'Gender analysis',
'Gender identity',
'Gender discrimination',
'Gender disparity',
'Male gender',
'Gender balance',
'Grammatical gender',
'Gender dysphoria',
'Gender mainstreaming',
'Gender diversity',
'Transgender Person',
'Gender inequality',
'Gender pay gap',
'Transgender people',
'Gender psychology',
'Gender Identity Disorder',
'Gender and development',
'Transgender women']

Fachsuche=['political science']

Fachsuche=['artificial intelligence',
'artificial intelligence system',
'music and artificial intelligence',
'artificial intelligence situated approach',
'applications of artificial intelligence',
'symbolic artificial intelligence',
'marketing and artificial intelligence'
]

"0000-0001-5406-9458""0000-0001-5449-4593",

We use it to query the OpenAlex API for works that specified the ORCID URL within their metadata in the field '`authorships.author.orcid`'.
 Since the API uses [pagination](https://docs.openalex.org/api/get-lists-of-entities#pagination), we need to loop through all pages to get the complete result set.

In [4]:
# OpenAlex endpoint to query for works
OPENALEX_API_WORKS = "https://api.openalex.org/works"

# query all works that are connected to orcid
def query_openalex_for_person2works(orcid):
    page = 1
    max_page = 1
    
    while page <= max_page:
        params = {'filter': 'authorships.author.orcid:'+orcid, 'page': page}
        response = requests.get(url=OPENALEX_API_WORKS,
                                params=params,
                                headers= {'Accept': 'application/json'})
        response.raise_for_status()
        result=response.json()

        # calculate max page number in first loop
        if max_page == 1:
            max_page = determine_max_page(result)
        page = page + 1
        yield result

# calculate max number of result pages
def determine_max_page(response_data):
    item_count = response_data['meta']['count']
    items_per_page = response_data['meta']['per_page']
    max_page_ceil = item_count // items_per_page + bool(item_count % items_per_page)
    return max_page_ceil


# ---- example execution
# list_of_pages=query_openalex_for_person2works(example_orcid)

From the resulting list of works we extract and print out title and DOI. 

*Note: works that do not have a DOI assigned, will not be printed.*

In [5]:
# from the result pages we get from the OpenAlex API, extract the data about works
def extract_works_from_page(page):
    return [work for work in page.get('results') or []]

# extract DOI from work
def extract_doi(work):
    doi=work.get('ids', {}).get('doi') or ""
    doi_id=doi.replace("https://doi.org/", "") if doi else doi
    title=work.get('display_name', "")
    concept=work.get('concepts')
    return doi_id, title, concept

def main_search(orcid):
    global Error_count
    # Abfrage der DOI Liste
    result_doi=[]
    count_doi=0
    list_of_pages=query_openalex_for_person2works(orcid)
    for page in list_of_pages or []:
        works=extract_works_from_page(page)
        for work in works or []:
            doi,title,concept=extract_doi(work)
            if doi:
                add=[]
                add.append(orcid)
                add.append(doi)
                add.append(title)
                add_concept=[]
                for item in concept:
                # Sublevel einfügen, am Besten alle Level!
                    all_concepts=[item['display_name'],'Level:'+str(item['level']),item['score']]
                    add_concept.append(all_concepts)
                add.append(add_concept)
                result_doi.append(add)
    # Beginn der Expertensuche            
    dict_gesamt={}
    dict_gesamt.update({'ID':orcid})
    dict_gesamt.update({'Count DOI:':count_doi})
    add=[]
    dedub_add=[]
    # Erstellen einer Liste von concepts aus der DOI Liste
    for item in result_doi:
        if orcid in item:
            count_doi=count_doi+1
            for item2 in item[3]:
                new=item2[0]
                add.append(new) 
        dict_gesamt.update({'Count DOI:':count_doi})
    # Dedublizierung der concepts liste
    for item in add:
        if item not in dedub_add:
            dedub_add.append(item)
    # Scores für jedes concept entwickeln
    for single_concept in dedub_add:
        score_concept=0
        concept_count=0
        for item in result_doi:
            for item2 in item[3]:
                if single_concept in item2[0]:
                    score_concept=score_concept+float(item2[2])
                    concept_count=concept_count+1
            if concept_count>0:
                final_score=score_concept/concept_count
                dict_gesamt.update({single_concept:final_score}) 
    print('#######',dict_gesamt)
    # Fehlersuche. Suche nach unmöglichen scores.
    dict_error=dict_gesamt.copy()
    del dict_error['ID']
    del dict_error['Count DOI:']
    error_check=dict_error.values()
    for item in error_check:
        if item >1:
            Error_count=Error_count+1
            print('############Error#############')
    # Abfrage nach Experten
    check=0
    expert=['### Orcid:', dict_gesamt['ID']]
    for item in Fachsuche:
        if item in dict_gesamt.keys() and dict_gesamt[item]>0.3:
            check=1
            print('Fach:', item,'Experte:',dict_gesamt['ID'],'Score:',dict_gesamt[item])
            add=['Fach:'+item,'Score:', dict_gesamt[item]]
            expert.append(add)
    if check ==1:
        list_experts.append(expert)

In [6]:
# main programm:
global Error_count
Error_count=0
global list_experts
list_experts=[]
for item in list_of_ids:
    main_search(item) 

####### {'ID': '0000-0001-5380-4449', 'Count DOI:': 0}
####### {'ID': '0000-0001-5406-9458', 'Count DOI:': 765, 'Patch testing': 0.6253272506249999, 'Guideline': 0.7362207745454544, 'Medicine': 0.6589606989424861, 'Allergic contact dermatitis': 0.6014122389361701, 'Judgement': 0.6657629, 'Contact dermatitis': 0.41593715679207915, 'Dermatology': 0.500206344909091, 'Contact allergy': 0.6206845755555555, 'Patch test': 0.6500741361764705, 'Continuing education': 0.45625195, 'Checklist': 0.41846994, 'Allergy': 0.311672148677966, 'Pathology': 0.12373654888, 'Medical education': 0.34824021071428574, 'Immunology': 0.17228507888805972, 'Psychology': 0.35484831477272727, 'Political science': 0.1924850536263736, 'Law': 0.0330778413, 'Cognitive psychology': 0.21010566, 'Psoriasis': 0.632507667142857, 'Epidemiology': 0.5091608088888889, 'Incidence (geometry)': 0.5707510023529412, 'Systematic review': 0.5776172159999999, 'Disease': 0.28945653038461544, 'MEDLINE': 0.4300469694736842, 'Family medicine

####### {'ID': '0000-0001-5449-4593', 'Count DOI:': 39, 'Formins': 0.7833252725000001, 'Biology': 0.7294314473529413, 'Hypha': 0.6118398024999999, 'Cell biology': 0.566512068, 'Tip growth': 0.6130495499999999, 'Morphogenesis': 0.560164025, 'Mutant': 0.5114800594444444, 'Cell polarity': 0.57165189, 'Actin': 0.43943723461538453, 'Fungal protein': 0.4894607766666667, 'Actin cytoskeleton': 0.36688381000000003, 'Cytoskeleton': 0.30493813166666667, 'Botany': 0.187420165, 'Genetics': 0.3458287555, 'Gene': 0.3173493695, 'Pollen': 0.0451466675, 'Pollination': 0.0, 'Cell': 0.4282692536666666, 'Pollen tube': 0.090293335, 'Kluyveromyces lactis': 0.8720585062499999, 'Saccharomyces cerevisiae': 0.62520654875, 'Complementation': 0.6744430774999999, 'Cyclin-dependent kinase 7': 0.5293549, 'Biochemistry': 0.46405856529411765, 'Open reading frame': 0.48406783, 'Protein-fragment complementation assay': 0.46944438, 'Kinase': 0.44833128666666666, 'Protein kinase A': 0.4279425075, 'Peptide sequence': 0.3703

####### {'ID': '0000-0001-7263-2670', 'Count DOI:': 0}
####### {'ID': '0000-0001-7364-4315', 'Count DOI:': 219, 'Big Five personality traits': 0.45550402500000003, 'Psychology': 0.45372813262711864, 'Personality': 0.5863214957142857, 'Social psychology': 0.37966965333333336, 'Big Five personality traits and culture': 0.37502125, 'Openness to experience': 0.4192037, 'Erg': 0.92311966, 'TMPRSS2': 0.7636888, 'Prostate cancer': 0.7264112099999999, 'Prostate': 0.678376326, 'Biology': 0.4294132972063491, 'Cancer research': 0.46621268461538456, 'Immunohistochemistry': 0.45040149333333335, 'Internal medicine': 0.4031191038611111, 'Cancer': 0.4656462104347826, 'Medicine': 0.3651578458799999, 'Oncology': 0.4326029033333334, 'Pathology': 0.354373628, 'Retina': 0.0, 'Disease': 0.16771158142857143, 'Coronavirus disease 2019 (COVID-19)': 0.18269354333333332, 'Neuroscience': 0.3682771953846154, 'Infectious disease (medical specialty)': 0.03939102, 'Epilepsy': 0.729501185, 'Hippocampal sclerosis': 0.6

####### {'ID': '0000-0001-7389-8024', 'Count DOI:': 0}
####### {'ID': '0000-0001-7973-3140', 'Count DOI:': 0}
####### {'ID': '0000-0001-8234-9166', 'Count DOI:': 34, 'Chemistry': 0.5249249039999999, 'Fluorescence correlation spectroscopy': 0.626950725, 'Flow (mathematics)': 0.54926825, 'Volume (thermodynamics)': 0.53433895, 'Micrometer': 0.52236225, 'Diffusion': 0.575246755, 'Resolution (logic)': 0.50353926, 'Beam (structure)': 0.5030491, 'Cross-correlation': 0.49335876, 'Volumetric flow rate': 0.4629439, 'SIGNAL (programming language)': 0.4421293, 'Realization (probability)': 0.4167313, 'Biological system': 0.4072687, 'Analytical Chemistry (journal)': 0.391498985, 'Mechanics': 0.34688726000000003, 'Optics': 0.3336846633333333, 'Molecule': 0.33229767, 'Physics': 0.08647695333333333, 'Chromatography': 0.101107522, 'Statistics': 0.05321270333333333, 'Mathematics': 0.22808835970588237, 'Organic chemistry': 0.0, 'Quantum mechanics': 0.0, 'Artificial intelligence': 0.209244672, 'Biology': 0

####### {'ID': '0000-0001-8343-8654', 'Count DOI:': 62, 'Competence (human resources)': 0.74756531, 'Mathematics education': 0.5220184555555556, 'Foundation (evidence)': 0.5218279, 'Psychology': 0.4242337627777777, 'Pedagogy': 0.35122223999999996, 'Social psychology': 0.18952994, 'Political science': 0.45821162473684207, 'Law': 0.0, 'Math wars': 0.47375375, 'Reform mathematics': 0.47195235, 'Professional development': 0.4595645, 'Connected Mathematics': 0.45309618, 'Core-Plus Mathematics Project': 0.42429015, 'Mathematics': 0.3880150240555556, 'Computer science': 0.34316087444444443, 'Calculus (dental)': 0.35793346, 'Medicine': 0.18788619333333334, 'Dentistry': 0.0, 'Content (measure theory)': 0.52822142, 'Early childhood': 0.48565848857142857, 'Developmental psychology': 0.22280416749999998, 'Mathematical analysis': 0.0, 'Philosophy': 0.28648837192307697, 'Humanities': 0.4532571605263158, 'Science education': 0.70070276, 'Flexibility (engineering)': 0.6943662, 'Empirical research': 0.

...

####### {'ID': '0000-0003-3459-5148', 'Count DOI:': 49, 'Unimodular matrix': 0.88366354, 'Polytope': 0.8774513650000001, 'Combinatorics': 0.46848715720000006, 'Mathematics': 0.8248748302083332, 'Lattice (music)': 0.51211977, 'Affine transformation': 0.6618162025, 'Quotient': 0.53998916, 'Discrete mathematics': 0.3728222110526315, 'Pure mathematics': 0.46673389045454544, 'Physics': 0.0033349611818181816, 'Acoustics': 0.0, 'Toeplitz matrix': 0.73422721, 'Generalization': 0.708893575, 'Eigenvalues and eigenvectors': 0.307281525, 'Measure (data warehouse)': 0.5584784, 'Simple (philosophy)': 0.47931539, 'Bounded function': 0.589172435, 'Multivariate statistics': 0.5385467, 'Applied mathematics': 0.45902799, 'Matrix (chemical analysis)': 0.48733366499999997, 'Unit circle': 0.4743513, 'Condition number': 0.4504222, 'Kernel (algebra)': 0.43812492, 'Mathematical analysis': 0.12355257575000002, 'Statistics': 0.0574220655, 'Philosophy': 0.030162641875, 'Materials science': 0.0, 'Epistemology': 0.

####### {'ID': '0000-0003-3654-5267', 'Count DOI:': 730, 'Visual cortex': 0.6156571401639345, 'Stimulus (psychology)': 0.6195288689130435, 'Neuroscience': 0.371001710673759, 'Orientation column': 0.5294153925, 'Synchronization (alternating current)': 0.5342349750000001, 'Phase synchronization': 0.455669915, 'Physics': 0.20265989532608686, 'Visual field': 0.5308687683333334, 'Biological system': 0.41049045071428564, 'Biology': 0.2850780809612069, 'Psychology': 0.4255876195285281, 'Striate cortex': 0.49126153833333336, 'Mathematics': 0.17917257822522523, 'Topology (electrical circuits)': 0.2425688288333333, 'Phase (matter)': 0.23315565, 'Cognitive psychology': 0.39318973274093255, 'Combinatorics': 0.015660743529411766, 'Quantum mechanics': 0.015510938968253968, 'Posterior parietal cortex': 0.52618464, 'Cortex (anatomy)': 0.51181185, 'Cerebral cortex': 0.5362784, 'Lag': 0.51422617, 'Temporal cortex': 0.5002723733333333, 'Cortical neurons': 0.48798104875000003, 'Computer science': 0.445767

####### {'ID': '0000-0003-4331-8695', 'Count DOI:': 168, 'Vacuole': 0.6977972866666665, 'Lipid bilayer fusion': 0.6194231916129033, 'Cell biology': 0.6445586788235297, 'Organelle': 0.570168573255814, 'Biology': 0.5887077280487804, 'SNARE complex': 0.5508854444999998, 'Vesicle': 0.48148379999999996, 'Vesicular Transport Proteins': 0.5355983568181819, 'Chemistry': 0.4050526554492753, 'Membrane': 0.26822389425600013, 'Biochemistry': 0.2890099628057555, 'Cytoplasm': 0.19930655585542165, 'Vacuolar protein sorting': 0.2850292844, 'Endosome': 0.7528617806250002, 'Rab': 0.8060094296226415, 'Biogenesis': 0.6219591242307693, 'GTPase': 0.5404745220555556, 'Lysosome': 0.6071656552173913, 'Tethering': 0.7217355669696971, 'Effector': 0.5302977, 'Protein targeting': 0.49914798500000007, 'Context (archaeology)': 0.51411367, 'Small GTPase': 0.5120712917391306, 'Vesicle fusion': 0.4716296566666667, 'Membrane protein': 0.32914087925925917, 'Signal transduction': 0.20365337307142856, 'Paleontology': 0.0, 

####### {'ID': '0000-0003-4939-1666', 'Count DOI:': 92, 'Environmental change': 0.54325661, 'Polar': 0.57953495, 'Ecology': 0.32659274010909084, 'Climate change': 0.60224256125, 'Northern Hemisphere': 0.51799476, 'Sensitivity (control systems)': 0.49361074, 'Southern Hemisphere': 0.48786592, 'Environmental science': 0.4924902002898553, 'Geography': 0.33307322147058827, 'Physical geography': 0.43841765560000007, 'Climatology': 0.373622452, 'Biology': 0.1469025124347826, 'Geology': 0.34414762875000005, 'Physics': 0.0191093675, 'Astronomy': 0.0, 'Electronic engineering': 0.0, 'Engineering': 0.062294155, 'Permafrost': 0.7597769666666667, 'Soil carbon': 0.6374366971428571, 'Database': 0.5367796, 'Soil map': 0.58326045, 'Pedogenesis': 0.703812975, 'Digital soil mapping': 0.47029493, 'Soil water': 0.3832479382857143, 'Soil survey': 0.4460539833333333, 'Soil science': 0.3091130761764706, 'Carbon cycle': 0.5249286566666668, 'Earth science': 0.38750653500000004, 'Computer science': 0.19669979353

####### {'ID': '0000-0003-4971-9991', 'Count DOI:': 74, 'Selfing': 0.813394887857143, 'Outcrossing': 0.7155422, 'Biology': 0.7742562719178084, 'Mating system': 0.57013272, 'Evolutionary biology': 0.4625391110526317, 'Population': 0.3284635753333333, 'Ecology': 0.38770372965517236, 'Mating': 0.41704911166666664, 'Pollen': 0.2318453566153846, 'Demography': 0.023287665826086962, 'Sociology': 0.004015078166666667, 'Population bottleneck': 0.6296188333333333, 'Ploidy': 0.57242212375, 'Pollination': 0.50453688, 'Genetic diversity': 0.44091852090909095, 'Genetics': 0.26683431507692307, 'Allele': 0.287796044, 'Microsatellite': 0.17630018, 'Gene': 0.291663512838983, 'Ecotype': 0.7044618233333333, 'Temperate climate': 0.5263109, 'Genetic variation': 0.43484776333333325, 'Botany': 0.46233481081632655, 'Polyploid': 0.7016613725, 'Genome': 0.4083757688333333, 'Genome evolution': 0.592289675, 'Brassicaceae': 0.7581198867647059, 'Taxon': 0.54715418, 'Phylogenetics': 0.45897886925, 'Plant ecology': 0.

In [7]:
print('######### Error Count:',Error_count, '#############')
print('# Anzahl der Experten:', len(list_experts))
print('# Expertenliste inkl. scores:',list_experts)

######### Error Count: 0 #############
# Anzahl der Experten: 5
# Expertenliste inkl. scores: [['### Orcid:', '0000-0001-8498-9466', ['Fach:Gender studies', 'Score:', 0.3556054433333333], ['Fach:Gender identity', 'Score:', 0.30286247], ['Fach:Gender Identity Disorder', 'Score:', 0.58766973]], ['### Orcid:', '0000-0002-1424-6314', ['Fach:Gender studies', 'Score:', 0.40501204428571425], ['Fach:Gender violence', 'Score:', 0.46264422]], ['### Orcid:', '0000-0002-6649-5064', ['Fach:Gender studies', 'Score:', 0.330563]], ['### Orcid:', '0000-0002-7366-679X', ['Fach:Gender studies', 'Score:', 0.466551615]], ['### Orcid:', '0000-0002-8722-3332', ['Fach:Gender studies', 'Score:', 0.38534521], ['Fach:Gender mainstreaming', 'Score:', 0.4916764]]]
